In [26]:
import collections
import os
import random
import tarfile
import torch
from torch import nn
import torchtext.vocab as Vocab
import torch.utils.data as Data
import csv
import sys
import math
import time
import json
import zipfile
from tqdm import tqdm
from PIL import Image
from collections import namedtuple
from IPython import display
from matplotlib import pyplot as plt
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchtext
import numpy as np
from itertools import chain
import torch.optim as optim
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim
import torch.autograd as autograd
import torchtext.vocab as torchvocab
from torch.autograd import Variable
import tqdm
import os
import time
import re
import pandas as pd
import string
import gensim
import time
import random
import snowballstemmer
import collections
from collections import Counter
from nltk.corpus import stopwords
from itertools import chain
from sklearn.metrics import accuracy_score



sys.path.append("..") 

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

DATA_ROOT = "/Users/shuxialin/Documents/project/code"

In [27]:
fname = os.path.join(DATA_ROOT, "aclImdb_v1.tar.gz")
if not os.path.exists(os.path.join(DATA_ROOT, "aclImdb")):
    print("从压缩包解压...")
    with tarfile.open(fname, 'r') as f:
        f.extractall(DATA_ROOT)

In [28]:
sentences = []
lines = []
with open("/Users/shuxialin/Documents/flip/01and03/data/se07_test.xml", 'r') as fileobj :
    for line in fileobj :
        lines.append(line)
        
lines = lines[1:]

for line in lines :
    tokens = (line.split('<')[1]).split('>')
    sentences.append(tokens[-1])

sen_se07 = sentences[:1000]

se07_classes = []

with open('/Users/shuxialin/Documents/flip/01and03/data/se07_test.emotions.gold', 'r') as flie :
    for line in flie:
        token = (line.split())[1:]
        a = token.index(max(token))
        se07_classes.append(a)
        
train_data = []
test_data = [] 

for i in range(800):
    train_data.append([sen_se07[i],se07_classes[i]])
for i in range(200):
    test_data.append([sen_se07[i+800],se07_classes[i+800]])

In [29]:
def get_tokenized_imdb(data):
    """
    data: list of [string, label]
    """
    def tokenizer(text):
        return [tok.lower() for tok in text.split(' ')]
    return [tokenizer(review) for review, _   in data]

In [30]:
def get_vocab_imdb(data):
    tokenized_data = get_tokenized_imdb(data)
    counter = collections.Counter([tk for st in tokenized_data for tk in st])
    return Vocab.Vocab(counter, min_freq=2)

vocab = get_vocab_imdb(train_data)
'# words in vocab:', len(vocab)

('# words in vocab:', 812)

In [31]:
def preprocess_imdb(data, vocab):
    max_l = 100  # 将每条评论通过截断或者补0，使得长度变成500

    def pad(x):
        return x[:max_l] if len(x) > max_l else x + [0] * (max_l - len(x))

    tokenized_data = get_tokenized_imdb(data)
    features = torch.tensor([pad([vocab.stoi[word] for word in words]) for words in tokenized_data])
    labels = torch.tensor([score for _, score in data])
    return features, labels

In [32]:
batch_size = 64
preprocess_imdb(train_data, vocab)
train_set = Data.TensorDataset(*preprocess_imdb(train_data, vocab))
test_set = Data.TensorDataset(*preprocess_imdb(test_data, vocab))
train_iter = Data.DataLoader(train_set, batch_size, shuffle=True)
test_iter = Data.DataLoader(test_set, batch_size)

In [33]:
class SentimentNet(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 bidirectional, labels, use_gpu, **kwargs):
        super(SentimentNet, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.num_layers = num_layers
        self.use_gpu = use_gpu
        self.bidirectional = bidirectional
        self.embedding = nn.Embedding(len(vocab), embed_size)
        self.embedding.weight.requires_grad = False
        self.encoder = nn.LSTM(input_size=embed_size, hidden_size=self.num_hiddens,
                               num_layers=num_layers, bidirectional=self.bidirectional,
                               dropout=0)
        if self.bidirectional:
            self.decoder = nn.Linear(num_hiddens * 4, labels)
        else:
            self.decoder = nn.Linear(num_hiddens * 2, labels)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        states, hidden = self.encoder(embeddings.permute([1, 0, 2]))
        encoding = torch.cat([states[0], states[-1]], dim=1)
        outputs = self.decoder(encoding)
        return outputs

In [34]:
num_epochs = 15
embed_size = 300
num_hiddens = 100
num_layers = 2
bidirectional = True
batch_size = 64
labels = 6
lr = 0.005
#device = torch.device('cuda:0')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
use_gpu = True

net = SentimentNet(vocab_size=(len(vocab)+1), embed_size=embed_size,
                   num_hiddens=num_hiddens, num_layers=num_layers,
                   bidirectional=bidirectional,
                   labels=labels, use_gpu=use_gpu)
net.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr)

In [36]:
def load_pretrained_embedding(words, pretrained_vocab):
    """从预训练好的vocab中提取出words对应的词向量"""
    embed = torch.zeros(len(words), pretrained_vocab.vectors[0].shape[0]) # 初始化为0
    oov_count = 0 # out of vocabulary
    for i, word in enumerate(words):
        try:
            idx = pretrained_vocab.stoi[word]
            embed[i, :] = pretrained_vocab.vectors[idx]
        except KeyError:
            oov_count += 0
    if oov_count > 0:
        print("There are %d oov words.")
    return embed

net.embedding.weight.data.copy_(
    load_pretrained_embedding(vocab.itos, glove_vocab))
net.embedding.weight.requires_grad = False # 直接加载预训练好的, 所以不需要更新它


In [44]:
result = []
loss_all = []
acc_all = []
for epoch in range(num_epochs):
    start = time.time()
    train_loss, test_losses = 0, 0
    train_acc, test_acc = 0, 0
    
    n, m = 0, 0
    for feature, label in train_iter:
        n += 1
        net.zero_grad()
        feature = Variable(feature)
        label = Variable(label)
        score = net(feature)
        loss = loss_function(score, label)
        #loss = nn.CrossEntropyLoss()
        loss.backward()
        optimizer.step()
        train_acc += accuracy_score(torch.argmax(score.cpu().data,
                                                 dim=1), label.cpu())
        train_loss += loss.cpu().item()
    with torch.no_grad():
        for test_feature, test_label in test_iter:
            m += 1
            test_feature = test_feature
            test_label = test_label
            test_score = net(test_feature)
            result.append(test_score)
            test_loss = loss_function(test_score, test_label)
            test_acc += accuracy_score(torch.argmax(test_score.cpu().data,
                                                    dim=1), test_label.cpu())
            test_losses += test_loss
    end = time.time()
    runtime = end - start
    loss_all.append(train_loss / n)
    acc_all.append(train_acc / n)
    print('epoch: %d, train loss: %.4f, train acc: %.2f, test loss: %.4f, test acc: %.2f, time: %.2f' %
          (epoch, train_loss / n, train_acc / n, test_losses.data / m, test_acc / m, runtime))



epoch: 0, train loss: 1.7418, train acc: 0.27, test loss: 1.7544, test acc: 0.22, time: 5.15
epoch: 1, train loss: 1.7311, train acc: 0.26, test loss: 1.7474, test acc: 0.22, time: 5.00
epoch: 2, train loss: 1.7190, train acc: 0.27, test loss: 1.7424, test acc: 0.22, time: 5.12
epoch: 3, train loss: 1.7106, train acc: 0.27, test loss: 1.7396, test acc: 0.22, time: 4.92
epoch: 4, train loss: 1.7031, train acc: 0.26, test loss: 1.7377, test acc: 0.22, time: 4.92
epoch: 5, train loss: 1.6935, train acc: 0.27, test loss: 1.7361, test acc: 0.22, time: 4.89
epoch: 6, train loss: 1.6951, train acc: 0.26, test loss: 1.7359, test acc: 0.22, time: 4.96
epoch: 7, train loss: 1.6765, train acc: 0.28, test loss: 1.7294, test acc: 0.23, time: 4.95
epoch: 8, train loss: 1.6636, train acc: 0.30, test loss: 1.7251, test acc: 0.23, time: 5.03
epoch: 9, train loss: 1.6449, train acc: 0.31, test loss: 1.7168, test acc: 0.23, time: 5.25
epoch: 10, train loss: 1.6297, train acc: 0.32, test loss: 1.7109, tes